---
code-fold: true
---



# Mapping El Corrido de Gregorio 

### packages required {.unnumbered}


In [ ]:
import folium
import geopandas as gpd
import pandas as pd

import re

from opencage.geocoder import OpenCageGeocode
import plotly.express as px 

import spacy
from spacy import displacy

### Extracting Name Enities (places) from text document {.unnumbered}


In [ ]:
text = "elcorridodegregoriocortez.txt"
with open(text, 'r', encoding='utf-8') as c:
    text = c.read()
    
def clean_text(text):
    cleaned= re.sub(r'[":;,.“”]', "", text)
    return(cleaned)
text = clean_text(text)
#print(text)


TxGPE=[]
nlp = spacy.load("tx_trained_ner")
doc =nlp(text) 
#print(doc)
for ent in doc.ents:
    #print(ent.text, ent.label_)
    if ent.label_ == "GPE":
        TxGPE.append(ent.text)
#print(TxGPE)

ents = [(e.text, e.start_char, e.end_char, e.label_)for e in doc.ents]
entsname=[(e.text) for e in doc.ents]
print(entsname)

### Visulization: Highlighting Name Entities from text {.unnumbered}


In [ ]:
displacy.render(doc, style ='ent', jupyter=False, page=True)

### Creating Dataframe {.unnumbered}


In [ ]:
df = pd.DataFrame({'NER': entsname})
geolocator = OpenCageGeocode(api_key='cee4174cec564ba5893e4a889de0b0b3')

geocode = lambda query: geolocator.geocode("%s, Texas" % query)
location = geolocator.geocode("Belmont Site", exactly_one=True)
df['Coordinates'] = df['NER'].apply(geocode)
df

### Creating GeoDataframe {.unnumbered}


In [ ]:
gdf = gpd.tools.geocode(df.Coordinates, provider='ArcGIS')
gdf = gpd.GeoDataFrame(gdf, crs="EPSG:4326")
gdf["lat"]=gdf['geometry'].y
gdf ["lon"] = gdf['geometry'].x

gdf
gdf.to_json()

### Creating Mapping Visualization from El Corrido de Gregorio Cortez {.unnumbered}


In [ ]:
# Ensure lat/lon columns exist
gdf["lat"] = gdf.geometry.y
gdf["lon"] = gdf.geometry.x

# Create a base map centered around the first coordinate or the mean location
m = folium.Map(location=[gdf["lat"].mean(), gdf["lon"].mean()], zoom_start=6, tiles="OpenStreetMap")

# Create a list of coordinate tuples (lat, lon)
coordinates = list(zip(gdf["lat"], gdf["lon"]))

# Add the line to the map
folium.PolyLine(
    locations=coordinates,
    color="blue",
    weight=3,
    opacity=0.7
).add_to(m)

# Add markers with hover text (optional)
for _, row in gdf.iterrows():
    folium.Marker(
        location=[row["lat"], row["lon"]],
        popup=row["address"],  # Shows address when clicked
        tooltip=row["address"]  # Shows address on hover
    ).add_to(m)

# Display the map (if running in a Jupyter Notebook)
m